In [1]:
cd /home/sam/bf-nn

/home/sam/bf-nn


In [2]:
import networkx as nx
from src.dataset import *
import torch
from torch.nn import Linear, Parameter, ReLU, LeakyReLU
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from src.models import *



In [9]:
class ResidualLayer(MessagePassing):
    def __init__(self, aggregation_config, update_config, act = 'ReLU' , **kwargs ):
        super(ResidualLayer, self).__init__()

        self.aggregation_layer = initialize_mlp(**aggregation_config)
         
        self.update_layer = initialize_mlp(**update_config)
        self.act = globals()[act]()

    def forward(self, x, edge_index, edge_attr):
        from_aggregation = self.propagate(edge_index, x=x, edge_attr=edge_attr)
        update_ = torch.cat((from_aggregation, x), dim=1)
        return self.act(self.update_layer(update_))
    
    def message(self, x_j, edge_attr):
        return self.act(self.aggregation_layer(torch.cat((x_j, edge_attr), dim=-1)))

In [10]:
aggregation_parameters = {'input': 2, 'hidden': 10, 'output': 5, 'layers': 4}
update_parameters = {'input': 10, 'hidden': 10, 'output': 6, 'layers': 2}

layer = ResidualLayer(aggregation_parameters, update_parameters)

In [11]:
G = construct_k_path(2, [2])
data = nx_to_bf_instance(G, 2, start = 1)

layer(data.x, data.edge_index, data.edge_attr)

torch.Size([2, 5])
torch.Size([2, 6])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x5 and 10x10)

In [2]:
model = BFModel(width=2, depth=2, bias=True, l0_regularizer=True)

In [5]:
G = construct_k_path(2, [2])
data1 = nx_to_bf_instance(G, 2, start = 1)
print(data1)

model(data1.x, data1.edge_index, data1.edge_attr)

BellmanFordStep(x=[2, 1], edge_index=[2, 4], edge_attr=[4, 1], y=[2, 1])


tensor([[23.6599],
        [23.6599]], grad_fn=<ReluBackward0>)

In [6]:
for layer in model.module_list:
    print(layer.get_l0_regularization_term())

tensor(2.3906, grad_fn=<AddBackward0>)
tensor(3.7470, grad_fn=<AddBackward0>)


In [31]:
class L0Linear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(L0Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.weight = Parameter(torch.Tensor(out_features, in_features))
        self.include_bias = bias
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        self.mu_weight = nn.Parameter(torch.normal(mean=torch.zeros((out_features, in_features)), 
                                                   std=torch.ones((out_features, in_features))))
        self.mu_bias = nn.Parameter(torch.normal(mean=torch.zeros(out_features), 
                                                 std=torch.ones(out_features)))
    
    def forward(self, x):
        if self.training:
            epsilon_weight = torch.normal(mean=torch.zeros((self.out_features, self.in_features)), 
                                          std = torch.ones((self.out_features, self.in_features)))
            epsilon_bias = torch.normal(mean=torch.zeros((self.out_features)), 
                                        std = torch.ones((self.out_features)))
        else: 
            epsilon_weight = torch.zeros((self.width, 2))
        z_weight = nn.functional.relu(torch.minimum(self.mu_weight + epsilon_weight, 
                                                    torch.ones((self.out_features, self.in_features))))
        z_bias = nn.functional.relu(torch.minimum(self.mu_bias + epsilon_bias, 
                                                    torch.ones(self.out_features)))
        out = torch.matmul(x, (self.weight * z_weight).T) + (self.bias * z_bias)
        return out

class SingleLayerArbitraryWidthBFLayer(MessagePassing):
    def __init__(self, 
                 width=2, 
                 bias=False, 
                 act='ReLU', 
                 l0_regularizer=False, 
                 **kwargs):
        super().__init__(aggr='min')
        if l0_regularizer:
            self.W_1 = L0Linear(in_features=2, out_features=2, bias=bias)
            self.W_2 = L0Linear(in_features=width, out_features=1, bias=bias)
        else:
            self.W_1 = Linear(in_features = 2, out_features = width, bias=bias)
            self.W_2 = Linear(in_features=width, out_features=1, bias=bias)

        # self.act = ReLU()
        self.act = globals()[act]()

        if bias:
            self.bias = True
        else:
            self.bias = False
    
    def random_init(self):
        torch.nn.init.normal_(self.W_1.weight, 0.0, 1.0)
        
        torch.nn.init.normal_(self.W_2.weight, 0.0, 1.0)
        
        if self.bias:
            torch.nn.init.normal_(self.W_1.bias, 0.0, 1.0)
            torch.nn.init.normal_(self.W_2.bias, 0.0, 1.0)

    
    def random_positive_init(self):
        torch.nn.init.uniform_(self.W_1.weight, a = 0.0, b=1.0)        
        torch.nn.init.uniform_(self.W_2.weight, a = 0.0, b=1.0)
        
        if self.bias:
            torch.nn.init.uniform_(self.W_1.bias, a = 0.0, b=1.0)
            torch.nn.init.uniform_(self.W_2.bias, a = 0.0, b=1.0)
    

    def forward(self, x, edge_index, edge_attr):
        return self.act(self.W_2(self.propagate(edge_index, x=x, edge_attr=edge_attr)))
    
    def message(self, x_j, edge_attr):
        return self.act(self.W_1(torch.cat((x_j, edge_attr), dim=-1)))

In [32]:
G = construct_k_path(2, [2])
data1 = nx_to_bf_instance(G, 2, start = 1)
print(data1)


BellmanFordStep(x=[2, 1], edge_index=[2, 4], edge_attr=[4, 1], y=[2, 1])


In [33]:
l0layer = SingleLayerArbitraryWidthBFLayer(width = 2, 
                                         bias=True, 
                                         act='ReLU', 
                                         l0_regularizer=True)

l0layer.fixed_init()

layer = SingleLayerArbitraryWidthBFLayer(width = 2, 
                                         bias=True, 
                                         act='ReLU', 
                                         l0_regularizer=False)
layer.fixed_init()


In [34]:
layer(data1.x, data1.edge_index, data1.edge_attr)

tensor([[3.],
        [7.]], grad_fn=<ReluBackward0>)

In [35]:
l0layer(data1.x, data1.edge_index, data1.edge_attr)

tensor([[0.],
        [0.]], grad_fn=<ReluBackward0>)

In [37]:
print(list(l0layer.parameters()))

[Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True), Parameter containing:
tensor([1., 1.], requires_grad=True), Parameter containing:
tensor([[-0.2874, -1.2583],
        [-0.1910,  0.7927]], requires_grad=True), Parameter containing:
tensor([-0.0817, -1.4638], requires_grad=True), Parameter containing:
tensor([[1., 1.]], requires_grad=True), Parameter containing:
tensor([1.], requires_grad=True), Parameter containing:
tensor([[-0.1801, -0.4578]], requires_grad=True), Parameter containing:
tensor([0.9373], requires_grad=True)]


In [25]:
vec = torch.tensor([[0, 1], [1, 1], [3, 1]], dtype=torch.float32)
l0linear = L0Linear(in_features=2, out_features=5)


In [26]:
l0linear(vec)

tensor([[2.7497e-34, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.5649e-41],
        [2.7497e-34, 1.7433e-09, 4.8864e-10, 1.3706e-09, 4.5649e-41],
        [2.7497e-34, 5.2298e-09, 1.4659e-09, 4.1118e-09, 4.5649e-41]],
       grad_fn=<AddBackward0>)

In [39]:
val = 0
assert val == 1

AssertionError: 

In [3]:
def three_node_path(edge_val, beta = 100):
    x = torch.tensor([[0], [edge_val], [beta]], dtype=torch.float)
    edge_index = torch.tensor([[0, 1, 1, 2, 0, 1, 2], 
                               [1, 0, 2, 1, 0, 1, 2]])
    edge_attr = torch.unsqueeze(torch.tensor([edge_val, edge_val, 0, 0, 0, 0, 0], dtype=torch.float), dim=1)
    y = torch.tensor([[0], [edge_val], [edge_val]], dtype=torch.float)
    return x, edge_index, edge_attr, y


In [13]:
def construct_m_path_dataset(m, sz=5):
    dataset = []
    for k in range(1, m):
        if k > 1:
            combinations = list(itertools.combinations(range(5*k), 2))
       # sz = len(combinations) if k > 1 else 2
        for i in range(sz):
            edge_weight = np.zeros(k)
            if k == 1:
                edge_weight[k - 1] = i
            else:
                edge_weight[0] = combinations[i][0]
                edge_weight[k - 1] = combinations[i][1]
            G = construct_k_path(k + 1, edge_weight, s=0)
            data = nx_to_bf_instance(G, k, start = 0)
            dataset.append(data)
    return dataset

In [14]:
data = construct_m_path_dataset(5)
print(data[2].x, data[2].y, data[2].edge_attr)


tensor([[  0.],
        [200.]]) tensor([[0.],
        [2.]]) tensor([[2.],
        [2.],
        [0.],
        [0.]])


In [3]:
print(G[0][1]['weight'])

NameError: name 'G' is not defined

In [129]:
import torch
from torch.nn import Linear, Parameter, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class BFLayer(MessagePassing):
    def __init__(self):
        super().__init__(aggr='min')
        self.W_1 = Linear(2, 1)
        self.relu = ReLU()
        self.w_2 = Parameter(torch.rand(1))
        self.b_2 = Parameter(torch.rand(1))

    def forward(self, x, edge_index, edge_attr):        
        return self.relu(self.w_2 * self.propagate(edge_index, x=x, edge_attr=edge_attr) + self.b_2)

    def message(self, x_j, edge_attr):

        return self.relu(self.W_1(torch.cat((x_j, edge_attr), dim=-1)))

In [130]:
nn_bf = BFLayer()


In [131]:
x = torch.tensor([[20], [0], [20]], dtype=torch.float)
edge_indx = torch.tensor([[0, 1, 1, 2, 0, 1, 2], [1, 0, 2, 1, 0, 1, 2]])
edge_attr = torch.unsqueeze(torch.tensor([5, 5, 10, 10, 0, 0, 0], dtype=torch.float), dim=1)

In [132]:
print(edge_attr)

tensor([[ 5.],
        [ 5.],
        [10.],
        [10.],
        [ 0.],
        [ 0.],
        [ 0.]])


In [133]:
nn_bf(x, edge_indx, edge_attr)

tensor([[ 5.],
        [ 0.],
        [10.]], grad_fn=<ReluBackward0>)